In [2]:
# Performs fast addition to evaluating [k]P by multiplying P by 2 each time and
# adding that result if the binary representation of k has a 1 in that spot
def fast_addition(E, P, k):
    out = E([0, 1, 0])
    if k == 0:
        return out
    r = int(log(k, 2))
    if k % 2 == 1:
        out += P
    for i in range(1, r + 1):
        P = P + P
        if int((k >> i) & 1) == 1:
            out += P
    return(out)

In [3]:
# This method is pretty slow so I end up opting for the faster prebuilt order() method
def find_elliptic_curve_order(p, a, b):
    order = 1
    for i in range(0, p):
        order += 1 + Jacobi(mod(i^3 + a*i + b, p), p)
    return order

In [4]:
# Finds the order of a point in an elliptic curve given the curve
# Relies on the theorem that g is a generator of the additive group G with identity element i iff 
# for every k factor of order(G), [order(G)/k] g != i. If it ever fails, the order of the point
# divides that l and thus we can recursively repeat that process.
def find_point_order(E, P, c_order):
    O = E([0, 1, 0])
    factors = [int(c_order/int(i[0])) for i in factor(c_order)]
    for l in factors:
        if fast_addition(E, P, l) == O:
            return find_point_order(E, P, l)
    return c_order

In [5]:
# Finds ind_P(Q) using the baby-step-giant-step algorithm
def ind(p, a, b, P, Q, verbose):
    # Curve Validation
    if not is_prime(p):
        if verbose: print("Error: " + str(p) + " is not prime...")
        return "Composite Characteristic Error"
    if (4*a^3 + 27*b^2 == 0):
        if verbose: print("Error: The discriminant is 0...")
        return "Zero Discriminant Error"
    if (mod(P[1]^2-P[0]^3-a*P[0]-b, p) != 0):
        if verbose: print("Error: P not on the curve...")
        return "Nonelement Error"
    # Generate the curve
    E = EllipticCurve(GF(p), [a, b])
    P = E([P[0], P[1], 1])
    # Checking if P is a generator
    E_order = E.order()
    P_order = find_point_order(E, P, E_order)
    if P_order != E_order:
        if verbose: print(str(P) + " is not a generator element...")
        return "Nongenerator Element Error"
    # Checking if Q is on the curve
    if (mod(Q[1]^2-Q[0]^3-a*Q[0]-b, p) != 0):
        if verbose: print("Error: Q not on the curve...")
        return "Nonelement Error"
    Q = E([Q[0], Q[1], 1])
    # Generating Baby List
    baby_l = []
    m = ceil(sqrt(P_order))
    if verbose: print("m = " + str(m))
    P_m = fast_addition(E, P, m)
    P_inv = E([P[0], -P[1], 1])
    sol_pair = [0, 0]
    for j in range(0, m):
        baby_l.append((j, Q + fast_addition(E, P_inv, j)))
    # Computing the left-hand-side until a match is found
    for i in range(0, m):
        U = fast_addition(E, P_m, i)
        for baby in baby_l:
            if U == baby[1]:
                if verbose: print("Found a match at i = " + str(i) + " and j = " + str(baby[0]) + ", where the left and right hand sides are equal to "  + str(U))
                sol_pair = [i, baby[0]]
                break
    x = sol_pair[0]*m + sol_pair[1]
    if verbose: print("Therefore, ind_P(Q) = im + j = " + str(x))
    if verbose:
        Q_computed = fast_addition(E, P, x)
        print(str(Q) + " = [" + str(x) +"] " + str(P) + " = " + str(Q_computed) + ". " + str(Q == Q_computed))
    return x # The index of Q sub P

In [6]:
# TESTING:

In [7]:
# Given Tests:
# Curve 1
p1 = 6917
a1 = 0
b1 = 6521
P1 = (210, 1338)
Q1 = (3329, 6379)
index1 = ind(p1, a1, b1, P1, Q1, True)
print("ind_P1(Q1) = " + str(index1))

# Curve 2
p2 = 6047
a2 = 0
b2 = 5986
P2 = (804, 2425)
Q2 = (4791, 105)
index2 = ind(p2, a2, b2, P2, Q2, True)
print("ind_P2(Q2) = " + str(index2))

# Curve 3
p3 = 5953
a3 = 0
b3 = 4687
P3 = (841, 5852)
Q3 = (5577, 235)
index3 = ind(p3, a3, b3, P3, Q3, True)
print("ind_P3(Q3) = " + str(index3))

m = 84
Found a match at i = 63 and j = 79, where the left and right hand sides are equal to (5585 : 2576 : 1)
Therefore, ind_P(Q) = im + j = 5371
(3329 : 6379 : 1) = [5371] (210 : 1338 : 1) = (3329 : 6379 : 1). True
ind_P1(Q1) = 5371
m = 78


Found a match at i = 64 and j = 6, where the left and right hand sides are equal to (2248 : 1598 : 1)
Therefore, ind_P(Q) = im + j = 4998
(4791 : 105 : 1) = [4998] (804 : 2425 : 1) = (4791 : 105 : 1). True
ind_P2(Q2) = 4998
m = 78
Found a match at i = 60 and j = 9, where the left and right hand sides are equal to (3950 : 5872 : 1)
Therefore, ind_P(Q) = im + j = 4689
(5577 : 235 : 1) = [4689] (841 : 5852 : 1) = (5577 : 235 : 1). True
ind_P3(Q3) = 4689


In [2]:
# Modified Jacobi where Jacobi(kp, p) = 0 For any k
def Jacobi(m, n):
    m = int(m)
    n = int(n)
    if mod(m,n) == 0:
        return 0
    temp = 1
    m -= int(m/n)*n
    if n % 2 == 0:
        print(str(n) + " is not an odd integer.")
        return
    if gcd(m, n) != 1:
        print(str(m) + " is not coprime to " + str(n))
        return
    while m % 2 == 0:
        temp *= pow(-1, (pow(n, 2) - 1)/8)
        m /= 2
    if m == 1:
        return temp * 1
    return temp * pow(-1, (m - 1)*(n - 1)/4) * Jacobi(n, m)

In [9]:
# Some Additional Tests:
for k in range(20):
    condition = True # The condition checks that p = 3 mod 4, disc != 0, and P is a generator
    while condition:
        condition = False
        p = random_prime(10^5, proof = True)
        condition = mod(p,4) == 1
        a = randint(1, p - 1)
        x = randint(1, p - 1)
        y = randint(1, p - 1)
        b = mod(y^2 - x^3 - a*x, p)
        condition = condition or 4*a^3 + 27*b^2 == 0
        E = EllipticCurve(GF(p),[a,b]);
        P = E([x, y ,1])
        condition = condition or E.order() != P.order()
    # Square root algorithm for the case when p = 3 mod 4
    m = p - 1
    while (not int(m) & 1):
        m = m/2
    repeat = True
    i = 0
    express = 0
    # We have to find a value for x_Q such that y_Q exists
    while repeat:
        i = randint(0, p-1)
        express = i^3 + a*i+b
        repeat = Jacobi(express, p) != 1
    ans = power_mod(express, (m + 1)/2 , p) # y_Q = ans
    P = (x, y)
    Q = (i, ans)
    print ("p: " + str(p) + " | a: " + str(a) + " | b: " + str(b) + " | P: " + str(P) + " | Q: " + str(Q))
    index = ind(p, a, b, P, Q, True)
    print("ind_P(Q) = " + str(index))
    print("#######################")

p: 66959 | a: 13340 | b: 58080 | P: (66083, 39177) | Q: (17785, 5018)
m = 259


Found a match at i = 43 and j = 197, where the left and right hand sides are equal to (666 : 48161 : 1)


Therefore, ind_P(Q) = im + j = 11334
(17785 : 5018 : 1) = [11334] (66083 : 39177 : 1) = (17785 : 5018 : 1). True
ind_P(Q) = 11334
#######################
p: 34939 | a: 22816 | b: 25295 | P: (20400, 4779) | Q: (14366, 13346)
m = 188


Found a match at i = 128 and j = 2, where the left and right hand sides are equal to (811 : 27532 : 1)
Therefore, ind_P(Q) = im + j = 24066
(14366 : 13346 : 1) = [24066] (20400 : 4779 : 1) = (14366 : 13346 : 1). True
ind_P(Q) = 24066
#######################
p: 61667 | a: 58935 | b: 52331 | P: (37097, 47822) | Q: (31269, 60520)
m = 248


Found a match at i = 201 and j = 153, where the left and right hand sides are equal to (50340 : 29993 : 1)
Therefore, ind_P(Q) = im + j = 50001
(31269 : 60520 : 1) = [50001] (37097 : 47822 : 1) = (31269 : 60520 : 1). True
ind_P(Q) = 50001
#######################


...

Therefore, ind_P(Q) = im + j = 20641
(45590 : 64805 : 1) = [20641] (75418 : 22796 : 1) = (45590 : 64805 : 1). True
ind_P(Q) = 20641
#######################
p: 54251 | a: 45709 | b: 20753 | P: (14284, 52137) | Q: (43534, 6383)
m = 233


Found a match at i = 89 and j = 133, where the left and right hand sides are equal to (50576 : 25590 : 1)


Therefore, ind_P(Q) = im + j = 20870
(43534 : 6383 : 1) = [20870] (14284 : 52137 : 1) = (43534 : 6383 : 1). True
ind_P(Q) = 20870
#######################
p: 80779 | a: 50168 | b: 17606 | P: (38397, 78031) | Q: (24095, 73355)
m = 285


Found a match at i = 15 and j = 226, where the left and right hand sides are equal to (68966 : 50769 : 1)


Therefore, ind_P(Q) = im + j = 4501
(24095 : 73355 : 1) = [4501] (38397 : 78031 : 1) = (24095 : 73355 : 1). True
ind_P(Q) = 4501
#######################


In [7]:
Jacobi(234512, 80779)

-1

In [6]:
is_prime(80779)

True